add data

In [2]:
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=4000)

word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'
x_train = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train])
x_test = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test])

1654784/1641221 [==============================] - 0s 0us/step


In [3]:
vocabulary = list()
for text in x_train:
  tokens = text.split()
  vocabulary.extend(tokens)

vocabulary = set(vocabulary)
print(len(vocabulary))

3998


In [4]:
from tqdm import tqdm

x_train_binary = list()
x_test_binary = list()

for text in tqdm(x_train):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_train_binary.append(binary_vector)

x_train_binary = np.array(x_train_binary)

for text in tqdm(x_test):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_test_binary.append(binary_vector)

x_test_binary = np.array(x_test_binary)

100%|██████████| 25000/25000 [06:56<00:00, 60.04it/s]


In [ ]:
print(x_test_binary[2])
print(y_test[2])
print(x_train_binary[2])
print(y_train[2])

[0 0 0 ... 0 0 0]
1
[0 0 0 ... 0 0 0]
0


Calculate entropy

In [5]:
import math
from math import log2 

def calculate_entropy_total_word(binary_array, c, word, word_position = -1):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth
  if word_position == -1:
    word_position = return_word(word) # the word column

  for element in binary_array:
      pos = element[word_position]
      count += 1
      if(pos == c):
        first += 1
      else:
        second += 1
  return -(first/count) * math.log2((first/count) - (second/count) * math.log2((second/count)))

def calculate_entropy_word(binary_array, c, word, word_position = -1):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth

  if word_position == -1:
    word_position = return_word(word) # the word column
  
  for element in binary_array:
      element = element[word_position]
      count += 1
      if(element == c):
        first += 1
      else:
        second += 1
  
  val = -(first/count) * log2((first/count) - (second/count) * log2((second/count)))
  return val


def calculate_entropy_total(binary_array, c):
  first = 0 # c value
  second = 0 # other than c value
  count = 0 # lengeth
  for element in binary_array:
      for pos in element:
        count += 1
        if(pos == c):
          first += 1
        else:
          second += 1
  return -(first/count) * log2((first/count) - (second/count) * log2((second/count)))


  

#print(calculate_entropy_total(x_train_binary, 0))  

return the position of a word

In [6]:
# -1 = has not found the word
def return_word(word):
  count = 0
  for element in vocabulary:
    if word == element:
      return count
    else:
      count += 1
  
  return -1

calculate probability

In [7]:
def calculate_probability_old(array, c, word, word_position = -1):
  count = 0
  if word_position == -1:
    word_position = return_word(word) # the word column
  for element in array:
    if(element[word_position] == c):
      count += 1
  return count / len(array)



def calculate_probability(array, c, word, word_position = -1):
  count = 0
  p = 0
  index = 0
  if word_position == -1:
    word_position = return_word(word) # the word column
  for element in array:
    if(element[word_position] == 1):
      count += 1
      if(y_train[index] == c):
        p += 1
    index += 1
  return p / count

calculate sum

In [8]:
def calculate_sum(binary_array, word):
  try:
    word_position = word.index
    p0 = (calculate_probability_old(binary_array, 0, word, word_position))
    p1 = (calculate_probability_old(binary_array, 1, word, word_position))
    word.setEntropy0(calculate_entropy_word(binary_array, 0, word,  word_position))
    word.setEntropy1(calculate_entropy_word(binary_array, 1, word,  word_position))
    a = p0 * word.entropy0
    b = p1 * word.entropy1
    return  a + b
  except:
    return None
  

calculate IG

In [9]:
def calculate_IG(binary_array, c, word, calculate_entrupy_total = 'no value'):
  if calculate_entrupy_total == 'no value':
    calculate_entrupy_total = calculate_entropy_total(binary_array, c)
  try:
    word.setIg(calculate_entrupy_total - calculate_sum(binary_array, word))
  except:
    word.setIg(None)

Get featured IG

We have some problem with 2 words


In [ ]:
import math
def get_featured_max_IG(binary_array, c):
  count = 0
  a = list()
  cet = calculate_entropy_total(binary_array, c)
  max = -9^999999999
  maxS = ""
  for x in vocabulary:
    count += 1
    try:
      result = calculate_IG(binary_array,1 , x, cet)
      if result > max:
        max = result
        maxS = x
      a.append(calculate_IG(binary_array,1 , x, cet))
    except:
      continue

  print("\n")
  print("\n")
  print("\n")
  return (result, maxS)

print(get_featured_max_IG(x_train_binary, 1))

Node

In [10]:
class Node:
  value: int
  p0: int
  p1: int
  left = None # must be Node
  right = None # must be Node

  def __init__(self, v: int = None):
    self.value = v
  
  def setLeftChild(self, v):
    self.left = v

  def setRightChild(self, v):
    self.right  = v

  def setChilds(self, l, r):
    self.setLeftChild(l)
    self.setRightChild(r)
  def result(self):
    if(p0 > p1):
      return "positive"
    return "negative"
  def __str__(self):
    return str(self.value)
      


h = Node(5)
h.setLeftChild(Node(3))
print(h)


    #create else

5


Remove words

In [13]:
def hasTheWord(word1: Word, word2: Word):
  index1 =word1.index
  index2 = word2.index
  for x in x_train_binary:
    if(x[index1] == 1 and x[index2] == 1):
      return 1

  return 0


In [12]:
class Word:

  def __init__(self, value:str, index:int):
    self.value = value
    self.index = index
    self.p0 = None
    self.p1 = None
    self.ig = None
    self.entropy0 = None
    self.entropy1 = None


  def setP0(self, p0):
    self.p0 = p0
  
  def setP1(self, p1):
    self.p1 = p1
  
  def setIg(self, ig):
    if(ig != None):
      self.ig = float(ig)

  def setEntropy0(self, entropy0):
    self.entropy0 = entropy0

  def setEntropy1(self, entropy1):
    self.entropy1  = entropy1

  def __str__(self):
    return self.ig

  def __repr__(self):
    return self.value + " " + str(self.ig)
  

In [14]:
#there is the word in the setence
def findValueFromArray(array, index):
  if len(array) < index:
    return None
  count = 0
  for x in array:
    if(count == index):
      return x
    count+= 1

In [15]:
def findTheNextWord(maxWord, array, c):
  nextWord = array.pop()
  while hasTheWord(maxWord, nextWord) != c:
    if len(array) == 0:
      return None
    try:
      nextWord = array.pop()
    except:
      return None
  return nextWord

Main run one

In [16]:
test = x_train_binary[0]
print(y_train[0])
ar = x_train_binary.tolist()
i = 0
et = calculate_entropy_total(x_train_binary, 1)
words = list()
for x in vocabulary:
  words.append(Word(x, i))
  i+= 1
words_to_remove = list()
for word in words:
  calculate_IG(x_train_binary, 1, word, et)
  word.setP0(calculate_probability(x_train_binary, 0, word, word.index))
  word.setP1(calculate_probability(x_train_binary, 1, word, word.index))
  if(word.ig == None):
    words_to_remove.append(word)



for x in words_to_remove:
  words.remove(x)



words.sort(key=lambda x: x.ig, reverse=True)
max = words.pop()
stopTheWhile = False
#next = findTheNextWord(max, words, c)
while len(words) !=0 and not stopTheWhile:
  c = findValueFromArray(test, max.index)
  while len(words) != 0 and max.p0 != 1 and max.p1 !=1:
    next = findTheNextWord(max, words, c)
    if(next == None):
      stopTheWhile = True
      break
    max = next


if(max.p0 > max.p1):
  print("bad")
  print(max.p0)
  print(max.p1)
else:
  print("good")
  print(max.p0)
  print(max.p1)



1


KeyboardInterrupt: ignored

create words

In [17]:
test = x_train_binary[0]
ar = x_train_binary.tolist()
i = 0
et = calculate_entropy_total(x_train_binary, 1)
words = list()
for x in vocabulary:
  words.append(Word(x, i))
  i+= 1
words_to_remove = list()
for word in words:
  calculate_IG(x_train_binary, 1, word, et)
  word.setP0(calculate_probability(x_train_binary, 0, word, word.index))
  word.setP1(calculate_probability(x_train_binary, 1, word, word.index))
  if(word.ig == None):
    words_to_remove.append(word)



for x in words_to_remove:
  words.remove(x)



words.sort(key=lambda x: x.ig, reverse=True)

Run it all

In [ ]:
j = 0
#next = findTheNextWord(max, words, c)
good_result = 0
print("hello let's start")
for sentense in x_train_binary:
  copy_words = words.copy()
  max = copy_words.pop()
  stopTheWhile = False
  while len(copy_words) !=0 and not stopTheWhile:
    c = findValueFromArray(sentense, max.index)
    while len(copy_words) != 0 and max.p0 != 1 and max.p1 !=1:
      next = findTheNextWord(max, copy_words, c)
      if(next == None):
        stopTheWhile = True
        break
      max = next
  if(max.p0 > max.p1):
    if 0 == y_train[j]:
      good_result += 1
  else:
    if 1 == y_train[j]:
      good_result += 1

  j+=1
  if(j%100 == 0):
    print(good_result/j*100)



print("teliko")
print(good_result/j*100)


hello let's start
48.0
